# Env Variables

In [100]:
TWEETS_PATH = "../../Donald-Tweets!.csv"

# Librarys

In [101]:
import pandas as pd
import re
import random

# Loading Dataframe

In [102]:
tweets_df = pd.read_csv(TWEETS_PATH)

# Removing non text tweets

In [103]:
# Trailing chars remotion
def remove_trailings(tweet):
    return re.sub(r'\S*_\s*$', '', tweet)

In [104]:
# Separation of \n
def add_trailing_space(tweet):
    return re.sub(r'\n', ' \n ', tweet)

In [105]:
# URL Remotion

def remove_urls(tweet):
    return re.sub(r'http\S+', '', tweet)

In [106]:
def remove_retweets(tweet):
    return re.sub(r'RT @\S*:', '', tweet)

In [107]:
tweets_df = tweets_df[tweets_df["Type"] == "text"]
tweets_df["Tweet_Text"] = tweets_df["Tweet_Text"].apply(lambda x: x.strip())
tweets_df["Tweet_Text"] = tweets_df["Tweet_Text"].apply(add_trailing_space)
tweets_df["Tweet_Text"] = tweets_df["Tweet_Text"].apply(remove_urls)
tweets_df["Tweet_Text"] = tweets_df["Tweet_Text"].apply(remove_retweets)
tweets_df["Tweet_Text"] = tweets_df["Tweet_Text"].apply(remove_trailings)

# Building the transition Index and Sparse Transition Matrix

In [108]:
transition_matrix = {}
token_counter = 0
unique_token_counter = 0
tweet_starters = []

for tweet in tweets_df["Tweet_Text"]:
    
    split_tweet = tweet.split()
    for idx, token in enumerate(split_tweet):
        
        token_counter += 1 

        # First time looking for a key top level token
        if token not in transition_matrix:

            transition_matrix[token] = {}
            unique_token_counter += 1

            if idx == 0:
                if "@" not in token:
                    tweet_starters.append(token)


        # Next word lookup
        if idx + 1 < len(split_tweet):
            next_token = split_tweet[idx + 1]
            
            if next_token not in transition_matrix[token]:
                transition_matrix[token][next_token] = 0
            
            transition_matrix[token][next_token] += 1

        # Add end of tweet token state
        else:
            if "<EOT>" not in transition_matrix[token]:
                transition_matrix[token]["<EOT>"] = 0
            transition_matrix[token]["<EOT>"] += 1

# Token info

In [109]:
print(f"Transitions for Today: {transition_matrix['Today']}")
print(f"Unique Tokens: {unique_token_counter}")
print(f"Dataset Tokens: {token_counter}")
print(f"Tweet Starters: {len(tweet_starters)}")

Transitions for Today: {'we': 3, 'in': 1, 'I': 2, 'they': 1, 'will': 3, 'is': 3, 'proves': 1, 'did': 1, 'we,': 1, 'National': 1, 'polls,': 1, '"We': 1}
Unique Tokens: 19280
Dataset Tokens: 109465
Tweet Starters: 1893


# Test run before moving it into a proper python model

In [110]:
word_treshold = 20

first_word = random.choice(tweet_starters)
current_word = first_word

sentence = ""

print(first_word)

while current_word != "<EOT>":

    sentence += f" {current_word}"

    current_word_dict = transition_matrix[current_word]

    keys = [key for key in current_word_dict]
    values = [current_word_dict[key] for key in current_word_dict]

    current_word = random.choices(keys, weights=values)[0]

    if current_word == "<EOT>":
        break


print(sentence)




"@PATRIOT4657:
 "@PATRIOT4657: @megynkelly @realDonaldTrump Will be great interview tonight.
